In [1]:
from mewpy.simulation import get_simulator
from cobra.io import read_sbml_model
from cobra.flux_analysis import flux_variability_analysis
import pandas
from time import time
import cobra.test
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)
from cobra.util.solver import linear_reaction_coefficients

In [3]:
cobra_model = cobra.io.read_sbml_model('./iml1515.xml')

In [4]:
cobra_model2 = cobra_model.copy()
len(cobra_model2.genes)

1516

In [5]:
solution = cobra_model.optimize()  #isto é sem as condiçoes ambientais
linear_reaction_coefficients(cobra_model)
#mudar as consiçoes ambientais
cobra_model.reactions.get_by_id("EX_o2_e").upper_bound = 1000
cobra_model.reactions.get_by_id("EX_o2_e").lower_bound = 0
cobra_model.reactions.get_by_id("EX_glc__D_e").upper_bound = 1000
cobra_model.reactions.get_by_id("EX_glc__D_e").lower_bound = -15
cobra_model.reactions.get_by_id("BIOMASS_Ec_iML1515_core_75p37M").upper_bound = 0.8769972144269682
cobra_model.reactions.get_by_id("BIOMASS_Ec_iML1515_core_75p37M").lower_bound = 0.2660981251911765
linear_reaction_coefficients(cobra_model)
cobra_model.objective = {cobra_model.reactions.EX_ac_e: -1.0}
solution = cobra_model.optimize()  #isto é otimizado para a biomassa e EX_ac_e com as condiçoes ambientais do WT
print('**** ANTES KO *****')
print('solution.objective_value',solution.objective_value)
print('fluxo EX_ac_e',solution.fluxes['EX_ac_e'])
print('fluxo biomassa',solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

**** ANTES KO *****
solution.objective_value -13.025232770730536
fluxo EX_ac_e 13.025232770730536
fluxo biomassa 0.2660981251911765


In [6]:
cobra_model2 = cobra_model.copy()
flu = []
for i in range(100):  # Deleção dos 100 primeiros genes
    cobra_model2.genes[i].knock_out()
    print(cobra_model2.genes[i])
    gene = [cobra_model2.genes[i]]
    solution = cobra_model2.optimize()
    solution.objective_value
    print('***** DEPOIS KO ******')
    print('solution.objective_value',solution.objective_value)
    gene.append(solution.objective_value)
    print('fluxo EX_ac_e',solution.fluxes['EX_ac_e'])
    gene.append(solution.fluxes['EX_ac_e'])
    print('fluxo biomassa',solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])
    gene.append(solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])
    flu.append(gene)
    cobra_model2 = cobra_model.copy()
    

b2551


/usr/local/lib/python3.7/site-packages/cobra/util/solver.py:508: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


***** DEPOIS KO ******
solution.objective_value 0.2814009531954382
fluxo EX_ac_e 13.030119453722673
fluxo biomassa 0.2660981251911765
b0870
***** DEPOIS KO ******
solution.objective_value 4.061284677049204e-05
fluxo EX_ac_e 13.025191819442838
fluxo biomassa 0.2660981251911765
b3368
***** DEPOIS KO ******
solution.objective_value 0.00017801964575292744
fluxo EX_ac_e 13.025232770729984
fluxo biomassa 0.2660981251911765
b2436
***** DEPOIS KO ******
solution.objective_value -13.025232770729707
fluxo EX_ac_e 13.025232770729707
fluxo biomassa 0.2660981251911765
b0008
***** DEPOIS KO ******
solution.objective_value -13.025232770729707
fluxo EX_ac_e 13.025232770729707
fluxo biomassa 0.2660981251911765
b3500
***** DEPOIS KO ******
solution.objective_value -13.025232770729707
fluxo EX_ac_e 13.025232770729707
fluxo biomassa 0.2660981251911765
b2465
***** DEPOIS KO ******
solution.objective_value -13.025232770729707
fluxo EX_ac_e 13.025232770729707
fluxo biomassa 0.2660981251911765
b0945
***** DEP

b3187
***** DEPOIS KO ******
solution.objective_value 5.933988191766287e-05
fluxo EX_ac_e 13.025232770729984
fluxo biomassa 0.2660981251911765
b3222
***** DEPOIS KO ******
solution.objective_value -13.025232770729707
fluxo EX_ac_e 13.025232770729707
fluxo biomassa 0.2660981251911765
b3225
***** DEPOIS KO ******
solution.objective_value -13.025232770729707
fluxo EX_ac_e 13.025232770729707
fluxo biomassa 0.2660981251911765
b0126
***** DEPOIS KO ******
solution.objective_value -13.025232770729707
fluxo EX_ac_e 13.025232770729707
fluxo biomassa 0.2660981251911765
b4269
***** DEPOIS KO ******
solution.objective_value -13.025232770729707
fluxo EX_ac_e 13.025232770729707
fluxo biomassa 0.2660981251911765
b0339
***** DEPOIS KO ******
solution.objective_value -13.025232770729707
fluxo EX_ac_e 13.025232770729707
fluxo biomassa 0.2660981251911765
b3942
***** DEPOIS KO ******
solution.objective_value -13.025232770729707
fluxo EX_ac_e 13.025232770729707
fluxo biomassa 0.2660981251911765
b4006
*****

Ao fazer a deleção dos primeiros 100 genes podemos verificar que o fluxo de acetato extracelular mantem-se igual.

In [27]:
import pandas as pd

df1 = pd.DataFrame.from_records(flu)
df1.columns = ["KO gene", "solution.objective_value", "fluxo ex_ac_e", "fluxo biomassa"]
print(df1)

   KO gene  solution.objective_value  fluxo ex_ac_e  fluxo biomassa
0    b2551                  0.281401      13.030119        0.266098
1    b0870                  0.000041      13.025192        0.266098
2    b3368                  0.000178      13.025233        0.266098
3    b2436                -13.025233      13.025233        0.266098
4    b0008                -13.025233      13.025233        0.266098
..     ...                       ...            ...             ...
95   b0720                  0.286107      13.025233        0.266098
96   b1398                -13.025233      13.025233        0.266098
97   b1136                  0.286107      13.025233        0.266098
98   b0049                -13.025233      13.025233        0.266098
99   b2103                  0.000059      13.025233        0.266098

[100 rows x 4 columns]


### Verificação de genes especificamente relacionados com o acetato extracelular 

In [29]:
model = cobra_model.copy()
envcond = {'EX_glc__D_e': (-15,100000),
           'EX_o2_e': (0,1000)}  # condições ambientais
simul = get_simulator(model, envcond=envcond)
result = simul.simulate(method='pFBA')
print(result.data_frame)

     Reaction ID      Flux
0         CYTDK2  0.000000
1           XPPT  0.000000
2          HXPRT  0.000000
3          NDPK5  0.000000
4         SHK3Dr  0.101415
...          ...       ...
2707        MPTS  0.000000
2708       MOCOS  0.000000
2709     BMOGDS2  0.000000
2710      FESD2s  0.000000
2711      OCTNLL  0.000000

[2712 rows x 2 columns]


Verificamos que existem reações de consumo de acetato que envolvem os seguintes genes:b2296, b1849, b3115. 

In [42]:
model.reactions.ACKr

Reaction identifier,ACKr
Name,Acetate kinase
Memory address,0x07f67dd976790
Stoichiometry,ac_c + atp_c <=> actp_c + adp_c Acetate + ATP C10H12N5O13P3 <=> Acetyl phosphate + ADP C10H12N5O10P2
GPR,b2296 or b1849 or b3115
Lower bound,-1000.0
Upper bound,1000.0


In [41]:
model.genes.b2296

Gene identifier,b2296
Name,ackA
Memory address,0x07f67dd9d6990
Functional,True
In 1 reaction(s),ACKr


In [34]:
print('b2296' in simul.essential_genes) 
print('b1849' in simul.essential_genes)
print('b3115' in simul.essential_genes)

True
True
True


In [32]:
#primeiro confirmamos o nome do gene:
print('Nome do gene b2296:', cobra_model.genes.b2296.name)  # ackA
print('Nome do gene b1849:', cobra_model.genes.b1849.name)  # purT
print('Nome do gene b3115:', cobra_model.genes.b3115.name)  # tdcD

Nome do gene b2296: ackA
Nome do gene b1849: purT
Nome do gene b3115: tdcD


In [35]:
for i in range(len(cobra_model.genes)):
    if 'ackA' in cobra_model.genes[i].name:
        print('O índice do gene b2296 é:', i)
    if 'purT' in cobra_model.genes[i].name:
        print('O índice do gene b1849 é:', i)
    if 'tdcD' in cobra_model.genes[i].name:
        print('O índice do gene b3115 é:', i)

O índice do gene b3115 é: 183
O índice do gene b1849 é: 186
O índice do gene b2296 é: 187


Estes genes têm indices compreendidos entre 180 e 190, pelo que experimentamos proceder à sua deleção:

In [36]:
cobra_model3 = cobra_model.copy()
flu3 = []
for i in range(180,191):  # Deleção dos 100 primeiros genes
    cobra_model3.genes[i].knock_out()
    print(cobra_model3.genes[i])
    gene = [cobra_model3.genes[i]]
    solution = cobra_model2.optimize()
    solution.objective_value
    print('***** DEPOIS KO ******')
    print('solution.objective_value',solution.objective_value)
    gene.append(solution.objective_value)
    print('fluxo EX_ac_e',solution.fluxes['EX_ac_e'])
    gene.append(solution.fluxes['EX_ac_e'])
    print('fluxo biomassa',solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])
    gene.append(solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])
    flu3.append(gene)
    cobra_model3 = cobra_model.copy()

b0908
***** DEPOIS KO ******
solution.objective_value -13.025232770729707
fluxo EX_ac_e 13.025232770729707
fluxo biomassa 0.2660981251911765
b1263
***** DEPOIS KO ******
solution.objective_value -13.025232770729707
fluxo EX_ac_e 13.025232770729707
fluxo biomassa 0.2660981251911765
b2458
***** DEPOIS KO ******
solution.objective_value -13.025232770729707
fluxo EX_ac_e 13.025232770729707
fluxo biomassa 0.2660981251911765
b3115
***** DEPOIS KO ******
solution.objective_value -13.025232770729707
fluxo EX_ac_e 13.025232770729707
fluxo biomassa 0.2660981251911765
b1264
***** DEPOIS KO ******
solution.objective_value -13.025232770729707
fluxo EX_ac_e 13.025232770729707
fluxo biomassa 0.2660981251911765
b2599
***** DEPOIS KO ******
solution.objective_value -13.025232770729707
fluxo EX_ac_e 13.025232770729707
fluxo biomassa 0.2660981251911765
b1849
***** DEPOIS KO ******
solution.objective_value -13.025232770729707
fluxo EX_ac_e 13.025232770729707
fluxo biomassa 0.2660981251911765
b2296
***** D

In [26]:
import pandas as pd

df = pd.DataFrame.from_records(flu3)
df.columns = ["KO gene", "solution.objective_value", "fluxo ex_ac_e", "fluxo biomassa"]
print(df)

   KO gene  solution.objective_value  fluxo ex_ac_e  fluxo biomassa
0    b0908                -13.025233      13.025233        0.266098
1    b1263                -13.025233      13.025233        0.266098
2    b2458                -13.025233      13.025233        0.266098
3    b3115                -13.025233      13.025233        0.266098
4    b1264                -13.025233      13.025233        0.266098
5    b2599                -13.025233      13.025233        0.266098
6    b1849                -13.025233      13.025233        0.266098
7    b2296                -13.025233      13.025233        0.266098
8    b2600                -13.025233      13.025233        0.266098
9    b2329                -13.025233      13.025233        0.266098
10   b3633                -13.025233      13.025233        0.266098


### Deleção de genes com outra estratégia:

In [50]:
cobra_model4 = cobra_model.copy()

solution = cobra_model2.optimize()

cobra_model4.reactions.get_by_id("EX_o2_e").upper_bound = 1000
cobra_model4.reactions.get_by_id("EX_o2_e").lower_bound = 0
cobra_model4.reactions.get_by_id("EX_glc__D_e").upper_bound = 1000
cobra_model4.reactions.get_by_id("EX_glc__D_e").lower_bound = -15
  
single_gene_deletion(
   cobra_model4, cobra_model4.genes[100:200], return_frame=True) 

,ids,growth,status
0,{b4151},-13.025233,optimal
1,{b2296},-13.025233,optimal
2,{b3633},NaN,infeasible
3,{b0071},NaN,infeasible
4,{b3925},-13.025233,optimal
...,...,...,...
95,{b2290},-13.025233,optimal
96,{b4384},NaN,infeasible
97,{b1263},NaN,infeasible
98,{b0072},NaN,infeasible
